In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as dsets
from skimage import transform
import torchvision.transforms as transforms
from torch.autograd import Variable
import pandas as pd;
import numpy as np;
from torch.utils.data import Dataset, DataLoader
#from vis_utils import *
import random;
import math;
import matplotlib.pyplot as plt

import os
print(os.listdir("../input"))

['train.csv', 'sample_submission.csv', 'test.csv']


In [ ]:
sample = pd.read_csv("../input/sample_submission.csv") 
sample.head()


In [ ]:
test = pd.read_csv("../input/test.csv") 
test.head()

In [ ]:
test.shape[0]

In [ ]:
train = pd.read_csv("../input/train.csv") 
train.head()


In [3]:
print(torch.__version__)
!python --version

1.0.0
Python 3.6.6 :: Anaconda, Inc.


In [4]:
class Dataset(object):
    """An abstract class representing a Dataset.
    All other datasets should subclass it. All subclasses should override
    ``__len__``, that provides the size of the dataset, and ``__getitem__``,
    supporting integer indexing in range from 0 to len(self) exclusive.
    """

    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __add__(self, other):
        return ConcatDataset([self, other])

In [5]:
class digitDataset(Dataset):
    
    def __init__(self, file_path, transform=None):
        self.data = pd.read_csv(file_path)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # in this example, i don't use ToTensor() method of torchvision.transforms
        # so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)
        image = self.data.iloc[index, 1:].values.astype(np.uint8).reshape((1, 28, 28))
        label = self.data.iloc[index, 0]
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label

In [6]:
class digitDatasetTrain(Dataset):
    
    def __init__(self, file_path, transform=None):
        self.data = pd.read_csv(file_path)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # in this example, i don't use ToTensor() method of torchvision.transforms
        # so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)
        image = self.data.iloc[index, 0:].values.astype(np.uint8).reshape((1, 28, 28))
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image

In [7]:
train_dataset = digitDataset('../input/train.csv', transform=None)
test_dataset = digitDatasetTrain('../input/test.csv', transform=None)

In [6]:
train_dataset.__getitem__(0)
test_dataset.__getitem__(0)

array([[[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  10,  17,
          17,  17,  17,  81, 180, 180,  35,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 139, 253,
         253, 253, 253, 253, 253, 253,  48,   0,   0,   0,   0,   0,
           0,   0,   0

In [8]:
class NeuralNet(nn.Module):
    """A Neural Network with a hidden layer"""
    def __init__(self, input_size,hidden_size,output_size):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        output = self.layer1(x)
        output = self.relu(output)
        output = self.layer2(output)
        return output

In [9]:
num_epochs = 5;
batch_size = 100;
learning_rate = 0.01;

In [10]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True);



In [11]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=28000,
                                         shuffle=False);

In [12]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [13]:
#instance of the Conv Net
cnn = CNN();
#loss function and optimizer
criterion = nn.CrossEntropyLoss();
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate);

In [14]:
losses = [];
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.float())
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item());
        
        if (i+1) % 100 == 0:
            print ('Epoch : %d/%d, Iter : %d/%d,  Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.item()))

Epoch : 1/2, Iter : 100/420,  Loss: 0.0997
Epoch : 1/2, Iter : 200/420,  Loss: 0.1427
Epoch : 1/2, Iter : 300/420,  Loss: 0.1674
Epoch : 1/2, Iter : 400/420,  Loss: 0.0525
Epoch : 2/2, Iter : 100/420,  Loss: 0.0449
Epoch : 2/2, Iter : 200/420,  Loss: 0.0659
Epoch : 2/2, Iter : 300/420,  Loss: 0.0661
Epoch : 2/2, Iter : 400/420,  Loss: 0.1093


In [15]:
cnn

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)

In [16]:

for i, (images) in enumerate(test_loader):
        images = Variable(images.float())
        outputs = cnn(images)


In [17]:
print(outputs.size())
print(outputs.type())
print(outputs.dim())

torch.Size([28000, 10])
torch.FloatTensor
2


In [18]:
print(outputs[[27999]])
print(outputs[:2, :])
outarray = outputs.detach().numpy()
outarray[[0]]

tensor([[ -6.2277, -10.6795,  10.4767,  -3.5487,  -7.7489, -10.6818,  -9.2150,
          -6.2460,  -0.2216,  -5.5276]], grad_fn=<IndexBackward>)
tensor([[ -3.8901,  -8.1182,  12.3082,  -2.9647,  -9.7094, -10.3573,  -9.8307,
          -2.3700,  -3.4633,  -9.0963],
        [  8.2662, -12.6513,  -2.4283,  -5.9596, -12.9961,  -3.5955,  -2.5945,
          -4.6787,  -6.8719,  -3.9221]], grad_fn=<SliceBackward>)


array([[ -3.8901339,  -8.118201 ,  12.308232 ,  -2.9647377,  -9.709358 ,
        -10.357319 ,  -9.8307   ,  -2.3700376,  -3.4632611,  -9.096293 ]],
      dtype=float32)

In [27]:
outarray2 = outarray.argmax(1)

In [31]:
outarray2.shape

(28000,)

In [34]:
outarray2[0:10]

array([2, 0, 9, 9, 3, 7, 0, 3, 0, 3])

In [38]:
out3 = outarray2

In [39]:
out3[0:10]

array([ 3,  1, 10, 10,  4,  8,  1,  4,  1,  4])

In [90]:
y = np.arange(28000)
y1 = y + 1
print(y1[0:10])
print(out3[0:10])
out3.shape

[ 1  2  3  4  5  6  7  8  9 10]
[ 3  1 10 10  4  8  1  4  1  4]


(28000,)

In [91]:
preds = np.column_stack((y1,out3))
preds = preds.astype(int)
print(preds.size)
preds[:, :]

56000


array([[    1,     3],
       [    2,     1],
       [    3,    10],
       ...,
       [27998,     4],
       [27999,    10],
       [28000,     3]])

In [95]:
np.savetxt("submission1.csv", preds, header="ImageId,Label", delimiter=",",comments='', fmt='%i')